In [7]:
# importing libraries and packages

import numpy as np
import mediapipe as mp
import cv2

In [2]:
#connecting keypoints visuals
mp_drawing = mp.solutions.drawing_utils
#keypoint   detection
mp_pose = mp.solutions.pose

In [3]:
#sequential mapping of the landmarks with body parts
count = 0
for i in mp_pose.PoseLandmark:
    print(i)
    count += 1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32


In [5]:
# 3d points of the landmarks
def calc_angle(a,b,c):
    a = np.array([a.x,a.y])
    b = np.array([b.x,b.y])
    c = np.array([c.x,c.y])
    
    ab=np.subtract(a,b)
    bc=np.subtract(b,c)

    theta = np.arccos(np.dot(ab,bc)/(np.linalg.norm(ab)*np.linalg.norm(bc)))
    theta = 180 -180*theta/3.14
    return np.round(theta,2)

In [8]:
#flag which stores hand position
flag = None
count = 0
#video capture
cap = cv2.VideoCapture(0)
#initializing pose
pose = mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5)
while cap.isOpened():
    _, frame = cap.read()

    #recoloring the image to RGB
    image = cv2.cvtColor(frame,cv2.COLOR_BGR2RGB)
    image.flags.writeable = False

    #detection
    results = pose.process(image)

    #recoloring back to BGR
    image.flags.writeable = True
    image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)

    #extracting landmarks
    try:
        landmarks = results.pose_landmarks.landmark
        shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
        elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
        wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]

        #calculating angle
        angle = calc_angle(shoulder,elbow,wrist)

        #visualizing angle
        cv2.putText(image,str(angle),tuple(np.multiply([elbow.x,elbow.y],[640,480]).astype(int)),cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2,cv2.LINE_AA)

        #curl counter logic
        if angle > 160:
            flag = "down"
        if angle < 30 and flag == "down":
            count += 1
            flag = "up"
        cv2.putText(image,'COUNT: '+str(count),(50,50),cv2.FONT_HERSHEY_SIMPLEX,0.5,(0,0,255),2,cv2.LINE_AA)
    except:
        pass

    # setup status box
    cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
    cv2.putText(image, str(count), (10,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
    # render detections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2),
                            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                            )
    cv2.imshow('Biceps Curl Counter', image)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()